<a href="https://colab.research.google.com/github/leegyuhi/oooooooooooooh/blob/master/Seq2seq_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ⅱ. Sequence to Sequence
## 1. seq2seq 
시퀀스-투-시퀀스(Sequence-to-Sequence)는 입력된 시퀀스로부터 다른 도메인의 시퀀스를 출력하는 다양한 분야에서 사용되는 모델입니다. 예를 들어 챗봇(Chatbot)과 기계 번역(Machine Translation)이 그러한 대표적인 예인데, 입력 시퀀스와 출력 시퀀스를 각각 질문과 대답으로 구성하면 챗봇으로 만들 수 있고, 입력 시퀀스와 출력 시퀀스를 각각 입력 문장과 번역 문장으로 만들면 번역기로 만들 수 있습니다.

![seq2seq_01](https://user-images.githubusercontent.com/52767505/70228477-c3c21b00-1797-11ea-8140-840c0aee2ddb.png)

위의 그림은 seq2seq 모델로 만들어진 번역기가 'I love you'라는 영어 문장을 받아서, 'Nan nul saranghey(난 널 사랑해)'라는 한국어 문장을 출력하는 모습을 보여줍니다. 지금부터 seq2seq 모델 내부 구성을 알아보겠습니다.

![seq2seq_02](https://user-images.githubusercontent.com/52767505/70228479-c45ab180-1797-11ea-9e70-87c6e0077258.png)

seq2seq는 크게 인코더와 디코더로 구성되어 있습니다. 인코더는 입력 문장의 모든 단어들을 순차적으로 입력받은 뒤에 마지막에 이 모든 단어 정보들을 압축해서 하나의 벡터로 만듭니다. 이 벡터를 context vector라고 합니다. 입력 문장의 정보가 하나의 context vector로 모두 압축되면, 이 벡터를 디코더로 전송합니다. 디코더는 context vector를 받아서 번역된 단어를 한 개씩 순차적으로 출력합니다.

![seq2seq_03](https://user-images.githubusercontent.com/52767505/70228480-c45ab180-1797-11ea-986f-e71968240e26.png)

인코더와 디코더는 각각 여러개의 RNN 셀로 구성되어 있습니다. 입력 문장을 받는 RNN 셀을 인코더라고 하고, 출력 문장을 내보내는 RNN 셀을 디코더라고 합니다. 일반적으로 RNN 셀의 성능문제 때문에 LSTM 셀이나 GRU 셀을 사용합니다. 

우선 인코더에서 입력 문장은 단어 단위로 쪼개져서 RNN 셀 각각의 입력이 됩니다. 인코더의 마지막 RNN 셀의 은닉 상태를 디코더의 RNN 셀로 넘겨주는데, 이를 컨텍스트 벡터라고 합니다. 이렇게 만들어진 컨텍스트 벡터는 앞에서 받은 모든 입력 값의 데이터를 함축적으로 가지고 있습니다. 컨텍스트 벡터는 다시 디코더 RNN 셀의 첫번째 은닉 상태로 사용됩니다.

컨텍스트 벡터를 받은 디코더는 초기 입력으로 문장의 시작을 의미하는 ```<start> ```심볼을 받습니다. 디코더가 ```<start>``` 심볼을 받으면 현재 시점에서 등장할 확률이 높은 단어를 예측합니다. 디코더의 첫번째 RNN 셀은 예측된 단어를 다음 시점의 RNN 셀의 입력으로 보냅니다. 그다음 RNN 셀은 입력된 단어로부터 다시 다음에 올 단어를 예측합니다. 디코더는 이런 식으로 입력된 단어의 다음에 올 단어를 예측하고 다음 시점의 RNN 셀의 입력으로 넣는 과정을 반복합니다. 문장의 끝을 의미하는 ```<end>``` 심볼이 다음 단어로 예측될 때 까지 이 과정을 반복합니다.

## 2. Encoder

### 2.1 Encoder

![seq2seq_07](https://user-images.githubusercontent.com/52767505/70231538-ddfef780-179d-11ea-9483-67391f9d59f3.png)

인코더에서 각각의 RNN 셀이 받은 입력을 어떻게 다음 RNN 셀로 넘기는지 좀 더 자세히 알아보도록 하겠습니다. 

현재 시점(time step)을 t라고 할 때, RNN 셀은 t-1에서의 은닉 상태와 t에서의 입력 벡터를 입력으로 받고, t에서의 은닉 상태를 만듭니다. 이때 t에서의 은닉 상태는 바로 위에 또 다른 은닉층이나 출력층이 존재할 경우에는 위의 층으로 보내거나, 필요없으면 값을 무시할 수 있습니다. 그리고 RNN 셀은 다음 시점에 해당하는 t+1의 RNN 셀의 입력으로 현재 t에서의 은닉 상태를 입력으로 보냅니다.

RNN 챕터에서도 언급했지만, 이런 구조에서 현재 시점 t에서의 은닉 상태는 과거 시점의 동일한 RNN 셀에서의 모든 은닉 상태의 값들의 영향을 누적해서 받아온 값이라고 할 수 있습니다. 그렇기 때문에 앞서 우리가 언급했던 컨텍스트 벡터는 사실 인코더에서의 마지막 RNN 셀의 은닉 상태값을 말하는 것이며, 이는 입력 문장의 모든 단어 토큰들의 정보를 요약해서 담고있다고 할 수 있습니다.

### 2.2 Embedding Layer

텍스트를 컴퓨터가 이해하고, 효율적으로 처리하게 하기 위해서는 컴퓨터가 이해할 수 있도록 텍스트를 적절히 변환할 필요가 있습니다. 단어를 표현하는 방법에 따라서 텍스트 처리의 성능이 크게 달라지기 때문입니다. 텍스트를 변환하는 방법으로는 'Word Embedding', 'Word2Vec', 'Glove'가 일반적으로 사용되고 있습니다.

![seq2seq_05](https://user-images.githubusercontent.com/52767505/70232056-e277e000-179e-11ea-860f-c045d157a2fb.png)

Seq2seq에서는 텍스트를 숫자로 변환하는 워드 임베딩을 이용합니다. 워드 임베딩은 단어를 벡터로 표현하는 것을 말합니다. 즉, seq2seq에서 사용되는 모든 단어들은 워드 임베딩을 통해 임베딩 벡터로 표현된 임베딩 벡터입니다. 

![seq2seq_06](https://user-images.githubusercontent.com/52767505/70231575-f242f480-179d-11ea-9a48-fb3726c3b8be.png)

예를 들어 I, love, you라는 단어들에 대한 임베딩 벡터는 위와 같은 모습을 가집니다. 여기서는 그림으로 표현하고자 사이즈를 4로 했지만, 일반적으로 임베딩 벡터는 수백 개의 차원을 가질 수 있습니다.


## 3. Decoder

### 3.1 Teacher Forcing

![seq2seq_08](https://user-images.githubusercontent.com/52767505/70232055-e277e000-179e-11ea-9bc1-83b63383049c.png)

Seq2seq는 훈련 과정과 테스트 과정의 작동 방식이 조금 다릅니다. 앞에서 설명한 과정은 테스트 과정에 해당됩니다. 반면, 훈련 과정에서는 교사 강요(teacher forcing)라는 방법을 이용합니다. 교사 강요란 디코더에게 인코더가 보낸 컨텍스트 벡터와 실제 정답인 "<start> I love you"를 입력 받았을 때, "Nan nul saranghey <end>"가 나와야 된다고 정답을 알려주면서 훈련하는 것입니다.

교사 강요를 사용하는 이유는 이전 시점의 디코더 셀의 예측이 틀렸는데 이를 현재 시점의 디코더 셀의 입력으로 사용하면 현재 시점의 디코더 셀의 예측도 잘못될 가능성이 높기 때문입니다. 이는 연쇄 작용으로 디코더 전체의 예측을 어렵게하고, 이런 상황이 반복되면 훈련 시간이 느려집니다.

### 3.2 Decoder

디코더에서 RNN 셀이 입력을 다음 RNN 셀로 넘기는 방법은 인코더와 동일합니다. 단, 첫번째 은닉 상태의 값으로 인코더의 마지막 RNN 셀의 은닉 상태인 컨텍스트 벡터를 사용한다는 점이 다릅니다. 디코더의 첫번째 RNN 셀은 이 첫번째 은닉 상태의 값과, 현재 t에서의 입력값인 ```<start>```로부터, 다음에 등장할 단어를 예측합니다. 그리고 이 예측된 단어는 다음 시점인 t+1 RNN에서의 입력값이 되고, 이 t+1에서의 RNN 또한 이 입력값과 t에서의 은닉 상태로부터 t+1에서의 출력 벡터. 즉, 또 다시 다음에 등장할 단어를 예측하게 될 것입니다. 

### 3.3 Softmax Equation

![seq2seq_09](https://user-images.githubusercontent.com/52767505/70232845-a9d90600-17a0-11ea-8577-ef6b8fcd2707.png)

출력 단어로 나올 수 있는 단어들은 다양한 단어들이 있습니다. seq2seq 모델은 선택될 수 있는 모든 단어들로부터 하나의 단어를 골라서 예측해야 합니다. 이를 예측하기 위해 소프트맥스 함수를 이용합니다. 디코더에서 각 시점(time step)의 RNN 셀에서 출력 벡터가 나오면, 해당 벡터는 소프트맥스 함수를 통해 출력 시퀀스의 각 단어별 확률값을 반환하고, 디코더는 출력 단어를 결정합니다.

![seq2seq_10](https://user-images.githubusercontent.com/52767505/70232993-00464480-17a1-11ea-89f7-6881bcd3d6a4.png)


# 참고문헌
- Dzmitry Bahdanau, KyungHyun Cho, Yoshua Bengio, "Neural Machine Translation by Jointly Learning to Aligh and Translate", 2015
- "How to Use Tensorflow for Seq2seq Models (LIVE)", Youtube, 2017.3.30, https://www.youtube.com/watch?v=ElmBrKyMXxs
- Won Joon Yoo, [딥 러닝을 이용한 자연어 처리 입문], BY-NC-SA, 2019.12.03, 13-1)시퀀스-투-시퀀스